In [ ]:
%pip install -q transformers bitsandbytes peft datasets evaluate trl

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import torch
from datasets import load_dataset
from google.colab import userdata

access_token = userdata.get('HF_TOKEN')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_name = 'mistralai/Mistral-7B-V0.3'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
training_args = TrainingArguments(
    output_dir = './results',
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    learning_rate = 2e-4,
    fp16 = True,
    optim = 'paged_adamw_32bit',
    logging_steps = 10,
    max_grad_norm = 0.35,
    warmup_ratio = 0.03,
    max_steps = 100,
    save_steps = 100,
    lr_scheduler_type = 'constant'
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dataset = load_dataset("json" , data_files = 'outputs.jsonl', field="data", split="train")
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
def createPrompt(example):
    bos_token = '<s>'
    system_prompt = '[INST] You are a finance suggestion model and your role is to give finance related suggestions \n'
    input_prompt = f" {example['input']} [/INST]"
    output_prompt = f"{example['output']} </s>"

    return [bos_token + system_prompt + input_prompt + output_prompt]

In [ ]:
trainer = SFTTrainer(
    model = model,
    peft_config = peft_config,
    formatting_func = createPrompt,
    max_seq_length = 512,
    tokenizer = tokenizer,
    args = training_args,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
)

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

In [ ]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
save_model.save_pretrained('./results/model')

In [ ]:
lora_config = LoraConfig.from_pretrained('./results/model')
model = get_peft_model(model, lora_config)

In [ ]:
text = "Question: How to invest your passive income?"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
inputs = tokenizer(text, return_tensors='pt').to(device)
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))